In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install av

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.3/34.3 MB 49.7 MB/s eta 0:00:00


In [2]:
import os
import pickle
import numpy as np
import torch
import av
from collections import defaultdict
from tqdm import tqdm
from transformers import VivitImageProcessor, VivitForVideoClassification
from sklearn.metrics.pairwise import cosine_similarity

## **1. 학습 모델 불러오기**

In [29]:
from transformers import pipeline
from transformers import VivitImageProcessor

model_name = "kkumtori/vivit-b-16x2-kinetics400-0511-mediapipe" # 허깅링크 9차회의록

image_processor = VivitImageProcessor.from_pretrained(model_name)
video_cls = pipeline(model = model_name)
video_cls.image_processor = image_processor

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [30]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

## **2. functions**

In [4]:
def read_video_pyav(container, indices):
    frames = []
    container.seek(0)
    start_index = indices[0]
    end_index = indices[-1]
    for i, frame in enumerate(container.decode(video=0)):
        if i > end_index:
            break
        if i >= start_index and i in indices:
            frames.append(frame)
    return np.stack([x.to_ndarray(format="rgb24") for x in frames])

def sample_frame_indices(clip_len, frame_sample_rate, seg_len):
    converted_len = int(clip_len * frame_sample_rate)
    end_idx = np.random.randint(converted_len, seg_len)
    start_idx = end_idx - converted_len
    indices = np.linspace(start_idx, end_idx, num=clip_len)
    indices = np.clip(indices, start_idx, end_idx - 1).astype(np.int64)
    return indices

def load_word_dictionary(pkl_file_path):
    """단어 사전 불러오는 함수"""
    with open(pkl_file_path, 'rb') as f:
        word_dictionary = pickle.load(f)
    return word_dictionary

In [31]:
def process_video_files(folder_paths):
    """hidden layer 뽑는 함수"""
    image_processor = video_cls.image_processor
    model = video_cls.model
    model.to(device)
    feature_dict = defaultdict(list)

    for folder_path in folder_paths:
        # 폴더명을 클래스 이름으로 사용
        for class_label in tqdm(os.listdir(folder_path)):
            class_path = os.path.join(folder_path, class_label)
            if not os.path.isdir(class_path):
                continue

            # 클래스 별 폴더 내 모든 파일을 탐색
            for filename in os.listdir(class_path):
                if filename.endswith(".mp4"):
                    file_path = os.path.join(class_path, filename)
                    container = av.open(file_path)

                    # 32 프레임 샘플링
                    indices = sample_frame_indices(clip_len=32, frame_sample_rate=1, seg_len=container.streams.video[0].frames)
                    video = read_video_pyav(container=container, indices=indices)

                    # 비디오를 모델에 맞게 준비
                    inputs = image_processor(list(video), return_tensors="pt").to(device)

                    # 모델을 통한 전파
                    with torch.no_grad():
                        outputs = model(**inputs, output_hidden_states=True)
                        logits = outputs.logits
                        hidden_states = outputs.hidden_states
                        last_hidden = hidden_states[-1]

                    # 클래스별로 마지막 히든 레이어의 특징 저장
                    predictions = torch.argmax(logits, dim=-1)
                    for idx, prediction in enumerate(predictions):
                        last_hidden = last_hidden[idx].cpu().numpy()
                        # print(last_hidden.shape)
                        feature_dict[class_label].append(last_hidden[idx])
    return feature_dict

In [32]:
def predict_with_dictionary(feature_dict, word_dictionary):
    """단어 사전을 사용한 예측 함수"""
    total_videos = 0
    correct_predictions = 0

    for class_label, features in feature_dict.items():
        for pred_feature in feature_dict[class_label]:
            similarities = {}
            for word, word_feature in word_dictionary.items():
                similarity = cosine_similarity([pred_feature], [word_feature])[0][0]
                similarities[word] = similarity
            most_similar_word = max(similarities, key=similarities.get)
            print(f"단어(class_label): {class_label}, 예측(most_similar_word): {most_similar_word}")

            total_videos += 1
            if most_similar_word == class_label:
                correct_predictions += 1
        print('---------------------------------')
    print(correct_predictions, '/', total_videos)
    return correct_predictions / total_videos

## **3. 단어사전 불러오기**

In [47]:
# 단어 사전
pkl_file_path_max = '/content/drive/MyDrive/기컴비_텀프/code/feature_dictionary/pkl/M_max_pooled_features.pkl'
pkl_file_path_average = '/content/drive/MyDrive/기컴비_텀프/code/feature_dictionary/pkl/M_average_pooled_features.pkl'

word_dictionary_max = load_word_dictionary(pkl_file_path_max)
word_dictionary_average = load_word_dictionary(pkl_file_path_average)

In [48]:
for k in word_dictionary_max.keys():
    print(list(word_dictionary_max[k])==list(word_dictionary_average[k]))

True
True
True
True
True
True
True
True
True
True


ㄴ 이상하다.. ! 피클이 잘못 저장된 듯  . .

In [10]:
# all_features.pkl 에서 avg,max 다시 뽑기
pkl_file_path_all = '/content/drive/MyDrive/기컴비_텀프/code/feature_dictionary/pkl/M_all_features.pkl'
word_dictionary_all = load_word_dictionary(pkl_file_path_all)

In [27]:
tmp_max = dict()
tmp_avg = dict()
for k,v in word_dictionary_all.items():
    print(k)
    tmp_max[k]=np.array(v).max(axis=0)
    tmp_avg[k]=np.array(v).mean(axis=0)

붕대
구급차
의사
골절
쓰러지다
가렵다
배고프다
친구
병원
다리


In [28]:
for k in tmp_max.keys():
    print(list(tmp_max[k])==list(tmp_avg[k]))

False
False
False
False
False
False
False
False
False
False


## **4. 평가**

In [33]:
# 폴더 경로 설정 및 평가 함수 호출
folder_paths = ['/content/drive/MyDrive/기컴비_텀프/data/train_dataset/mediapipe/test']
all_features = process_video_files(folder_paths)

  0%|          | 0/10 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/feature_extraction_utils.py:141: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:274.)
  return torch.tensor(value)
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/conv.py:605: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv3d(
100%|██████████| 10/10 [02:46<00:00, 16.68s/it]


In [34]:
for k,v in all_features.items():
    print(k,len(v),v[0].shape)

붕대 10 (768,)
구급차 10 (768,)
의사 10 (768,)
골절 10 (768,)
쓰러지다 10 (768,)
가렵다 10 (768,)
배고프다 10 (768,)
친구 10 (768,)
병원 10 (768,)
다리 10 (768,)


In [35]:
# acc_max = predict_with_dictionary(all_features, word_dictionary_max)
acc_max = predict_with_dictionary(all_features, tmp_max)

단어(class_label): 붕대, 예측(most_similar_word): 구급차
단어(class_label): 붕대, 예측(most_similar_word): 구급차
단어(class_label): 붕대, 예측(most_similar_word): 다리
단어(class_label): 붕대, 예측(most_similar_word): 쓰러지다
단어(class_label): 붕대, 예측(most_similar_word): 붕대
단어(class_label): 붕대, 예측(most_similar_word): 병원
단어(class_label): 붕대, 예측(most_similar_word): 병원
단어(class_label): 붕대, 예측(most_similar_word): 다리
단어(class_label): 붕대, 예측(most_similar_word): 붕대
단어(class_label): 붕대, 예측(most_similar_word): 다리
---------------------------------
단어(class_label): 구급차, 예측(most_similar_word): 쓰러지다
단어(class_label): 구급차, 예측(most_similar_word): 쓰러지다
단어(class_label): 구급차, 예측(most_similar_word): 가렵다
단어(class_label): 구급차, 예측(most_similar_word): 병원
단어(class_label): 구급차, 예측(most_similar_word): 다리
단어(class_label): 구급차, 예측(most_similar_word): 구급차
단어(class_label): 구급차, 예측(most_similar_word): 다리
단어(class_label): 구급차, 예측(most_si

In [36]:
# acc_average = predict_with_dictionary(all_features, word_dictionary_average)
acc_average = predict_with_dictionary(all_features, tmp_avg)

단어(class_label): 붕대, 예측(most_similar_word): 구급차
단어(class_label): 붕대, 예측(most_similar_word): 구급차
단어(class_label): 붕대, 예측(most_similar_word): 붕대
단어(class_label): 붕대, 예측(most_similar_word): 쓰러지다
단어(class_label): 붕대, 예측(most_similar_word): 붕대
단어(class_label): 붕대, 예측(most_similar_word): 의사
단어(class_label): 붕대, 예측(most_similar_word): 의사
단어(class_label): 붕대, 예측(most_similar_word): 다리
단어(class_label): 붕대, 예측(most_similar_word): 의사
단어(class_label): 붕대, 예측(most_similar_word): 붕대
---------------------------------
단어(class_label): 구급차, 예측(most_similar_word): 쓰러지다
단어(class_label): 구급차, 예측(most_similar_word): 친구
단어(class_label): 구급차, 예측(most_similar_word): 골절
단어(class_label): 구급차, 예측(most_similar_word): 의사
단어(class_label): 구급차, 예측(most_similar_word): 가렵다
단어(class_label): 구급차, 예측(most_similar_word): 구급차
단어(class_label): 구급차, 예측(most_similar_word): 다리
단어(class_label): 구급차, 예측(most_similar_

In [37]:
print(f'max_pooling : {acc_max}')
print(f'average_pooling : {acc_average}')

max_pooling : 0.39
average_pooling : 0.44
